In [0]:
%pip install mlflow==2.17.2 cloudpickle==2.2.1 
# hardcode the ml 15.4 LTS libraries versions here - should move to env_manager='conda' for prod use-case instead
%pip install category-encoders==2.6.3 cffi==1.15.1 cloudpickle==2.2.1 databricks-automl-runtime==0.2.21 defusedxml==0.7.1 holidays==0.45 lightgbm==4.2.0 lz4==4.3.2 matplotlib==3.7.2 numpy==1.23.5 pandas==1.5.3 psutil==5.9.0 pyarrow==14.0.1 scikit-learn==1.3.0 scipy==1.11.1
%pip install azure-core azure-storage-file-datalake #for the display() in Azure only

#Registering python UDF to a SQL function
This is a companion notebook to load the wind turbine prediction model as a spark udf and save it as a SQL function
 
Make sure you add this notebook in your DLT job to have access to the `get_turbine_status` function. (Currently mixing python in a SQL DLT notebook won't run the python)


<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=1444828305810485&notebook=%2F01-Data-ingestion%2F01.2-DLT-Wind-Turbine-SQL-UDF&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2F01-Data-ingestion%2F01.2-DLT-Wind-Turbine-SQL-UDF&version=1">

In [0]:
import mlflow
mlflow.set_registry_uri('databricks-uc')
#                                                                                                                        Stage/version  
#                                                                                           Model name                          |        
#                                                                                               |                               |        
predict_maintenance_udf = mlflow.pyfunc.spark_udf(spark, "models:/dbdemosfs1224group5.dbdemos_iot_platform.dbdemos_turbine_maintenance@prod", "string")
spark.udf.register("predict_maintenance", predict_maintenance_udf)

### Setting up the DLT 

This notebook must be included in your DLT "libraries" parameter:

```
{
    "id": "95f28631-1884-425e-af69-05c3f397dd90",
    "name": "xxxx",
    "storage": "/demos/dlt/lakehouse_iot_wind_turbine/xxxxx",
    "configuration": {
        "pipelines.useV2DetailsPage": "true"
    },
    "clusters": [
        {
            "label": "default",
            "autoscale": {
                "min_workers": 1,
                "max_workers": 5
            }
        }
    ],
    "libraries": [
        {
            "notebook": {
                "path": "/Repos/xxxx/01.1-DLT-Wind-Turbine-SQL"
            }
        },
        {
            "notebook": {
                "path": "/Repos/xxxx/01.2-DLT-Wind-Turbine-SQL-UDF"
            }
        }
    ],
    "target": "retail_lakehouse_churn_xxxx",
    "continuous": false,
    "development": false
}
```